In [1]:
import spotipy
import json
import webbrowser
import pandas as pd

In [2]:
# Importing library for spotify

import urllib.request
import spotipy.util as util

In [3]:
# Importing geolocation library and getting user's location
import geocoder
userGeolocation = geocoder.ip('me')
#print(userGeolocation.latlng)

In [4]:
# open file with keys and set the path to your credentials JSON file
credentials = "SpotifyTokens.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)
    
# read the keys and assign each to a variable
# A redirect URI, or reply URL, is the location where the authorization server sends the user once the app has been 
# successfully authorized and granted an authorization code or access token.
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-read-currently-playing'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [5]:
trafficKey = username = api_tokens["trafficKey"]

# Getting traffic data from London
request = urllib.request.Request("http://dev.virtualearth.net/REST/v1/Traffic/Incidents/51,%20-0.489,%2051.686,%200.236?key=" + trafficKey)
response = urllib.request.urlopen(request)

In [6]:
fullTrafficData = json.loads(response.read())

In [7]:
sp = spotipy.Spotify(auth=token)

In [8]:
# Making a boolean to check if song is being played or not
playingMusic = False

my_current_track = sp.current_user_playing_track()
#print(current_track['item']['name'])

# my_current_track only seems to be of type dict when a song is being played
# So if its a dictionary, music is being played
if type(my_current_track) is dict:
    playingMusic = True

# If no song is being played, print "no song being played", otherwise print the artis nam and song being played
if playingMusic == False:
    print ("No song being played")
else:
    print(f"Listening to {my_current_track['item']['name']} by {my_current_track['item']['album']['artists'][0]['name']}")


No song being played


In [9]:
amountTrafficCases = fullTrafficData["resourceSets"][0]['estimatedTotal']
trafficData = fullTrafficData["resourceSets"][0]['resources']

In [10]:
congestionDescription = []
for x in trafficData:
    if x['type'] == 5:
        congestionDescription += x['description'].split(".")
#print(congestionDescription)

In [11]:
severeCongestionCoordinates = []
for x in trafficData:
    if x['type'] == 5 and x['severity'] == 4:
        severeCongestionCoordinates.append(x['point']['coordinates'])
#print(severeCongestionCoordinates)

In [12]:
#Theres an error in the data where one of the current lines does not have a title so its complaining if i print it outside of the loop

severeCongestionStreets = []

for x in trafficData:
    if x['type'] == 5 and x['severity'] == 4:
        severeCongestionStreets.append(x['title'])
        #print(x['title'])
        #print(severeCongestionStreets)
    

KeyError: 'title'

In [15]:
track_results = sp.search(q='stuck', type='track', limit=50)
tracks = track_results['tracks']['items']

# URI = uniform resource identifier
track_selection_list = []
track_art = []

for song in tracks:
    track_selection_list.append(song['uri'])
    track_art.append(song['album']['images'][0]['url'])

In [16]:
# Trying to create a playlist for each congestion coordinate

for x in range(len(severeCongestionCoordinates)):
    playlistCoordinateString = str(severeCongestionCoordinates[x])
    
    my_playlists = sp.user_playlist_create(user=username, name=playlistCoordinateString, public=True,
                                      description="Playlist made where a traffic jam is")
    results = sp.user_playlist_add_tracks(username, my_playlists['id'], track_selection_list)

print(results)

HTTP Error for POST to https://api.spotify.com/v1/users/Apr52wWaQJ83ONQ1iWfzuEpMuja7ZGkxdWG5prGe2QhiVaJa9AXnqNnrPRs_qok0/playlists with Params: {} returned 403 due to You cannot create a playlist for another user.


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/users/Apr52wWaQJ83ONQ1iWfzuEpMuja7ZGkxdWG5prGe2QhiVaJa9AXnqNnrPRs_qok0/playlists:
 You cannot create a playlist for another user., reason: None

In [17]:
#Hardcoding a coordinate that is in traffic
testyUserInTraffic = [51.518559, -0.218466]

hardcodedUserStatusInTraffic = False
myRealStatusInTraffic = False

# for in range loop with a hardcoded lat and long to equate to true
# probs need to hardcode the value again in the future if want to test again since live data might change
for x in range(len(severeCongestionCoordinates)):
    #print("I like", severeCongestionCoordinates[x])
    if testyUserInTraffic == severeCongestionCoordinates[x]:
        hardcodedUserStatusInTraffic = True

#print(hardcodedUserStatusInTraffic)


# for in range loop with my real geolocation
for x in range(len(severeCongestionCoordinates)):
    #print("I like", severeCongestionCoordinates[x])
    if userGeolocation.latlng == severeCongestionCoordinates[x]:
        myRealStatusInTraffic = True

#print(myRealStatusInTraffic)

In [18]:
# Creating a playlist for each of the traffic jams with the coordinates names

for x in range(len(severeCongestionCoordinates)):
    playlistCoordinateString = str(severeCongestionCoordinates[x])
    
    my_playlists = sp.user_playlist_create(user=username, name=playlistCoordinateString, public=True,
                                      description="Playlist made where a traffic jam is")
    results = sp.user_playlist_add_tracks(username, my_playlists['id'], track_selection_list)

print(results)

HTTP Error for POST to https://api.spotify.com/v1/users/Apr52wWaQJ83ONQ1iWfzuEpMuja7ZGkxdWG5prGe2QhiVaJa9AXnqNnrPRs_qok0/playlists with Params: {} returned 403 due to You cannot create a playlist for another user.


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/users/Apr52wWaQJ83ONQ1iWfzuEpMuja7ZGkxdWG5prGe2QhiVaJa9AXnqNnrPRs_qok0/playlists:
 You cannot create a playlist for another user., reason: None

In [40]:
print(my_playlists)

{'collaborative': False, 'description': 'Playlist made where a traffic jam is', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/4gdAeXWdUKyW5NFI9t24C2'}, 'followers': {'href': None, 'total': 0}, 'href': 'https://api.spotify.com/v1/playlists/4gdAeXWdUKyW5NFI9t24C2', 'id': '4gdAeXWdUKyW5NFI9t24C2', 'images': [], 'name': '[51.525313, -0.260287]', 'owner': {'display_name': 'Rocio', 'external_urls': {'spotify': 'https://open.spotify.com/user/pinguinarey'}, 'href': 'https://api.spotify.com/v1/users/pinguinarey', 'id': 'pinguinarey', 'type': 'user', 'uri': 'spotify:user:pinguinarey'}, 'primary_color': None, 'public': True, 'snapshot_id': 'MSw0YTg5YmIzOGE1NmM5ZThjZTIyMDA1NzFlZjViYTJmN2NlYzllZGEw', 'tracks': {'href': 'https://api.spotify.com/v1/playlists/4gdAeXWdUKyW5NFI9t24C2/tracks', 'items': [], 'limit': 100, 'next': None, 'offset': 0, 'previous': None, 'total': 0}, 'type': 'playlist', 'uri': 'spotify:playlist:4gdAeXWdUKyW5NFI9t24C2'}


In [41]:
# sp.start_playback(deviceID, None, track_selection_list)
webbrowser.open(my_playlists['external_urls']['spotify'])

True

['Tithepit Shaw Lane', 'Tithepit Shaw Lane', 'Philpots Lane', 'Philpots Lane', 'Dalmeny Road', 'Dalmeny Road', 'Summerhouse Lane', 'Summerhouse Lane', 'Mansion House Street', 'Mansion House Street', 'Lime Street', 'Turner Road', 'B272 / Rowan Road', 'B272 / Rowan Road', 'Coldharbour Road', 'Coldharbour Road', 'Ide Hill Road', 'Pretoria Road', 'Pretoria Road', 'Penwortham Road', 'Penwortham Road', 'Broadwater Road', 'Hillbrook Road', "Wix's Lane", 'Beclands Road', 'Beclands Road', 'Hungerford Road', "St George's Way", 'Lionel Road South', 'Euston Square', 'Lionel Road South', 'B411 / Inverness Place', 'station road', "Queen's Crescent", 'Fernleigh Road', 'Fernleigh Road', "Queen's Road", 'Whittington Road', 'East Hill Road', 'college road', 'college road', 'East Hill Road', 'Homecroft Gardens', 'Homecroft Gardens', "Baron's Place", 'Old Street', 'Lichfield Road', 'Holly Road', 'Holly Road', 'Saint Helens Gardens', 'Mill Lane', 'Mill Lane', "Parker's Hill", "Parker's Hill", 'Tolworth Roa